### ML4NLP Ex03 - CNNs

In [2]:
import os
import re
import io
import requests
import torch
import numpy as np
import pandas as pd
import spacy
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
!pip install torchmetrics
from torchmetrics.classification import BinaryF1Score
import torch.optim as optim
from torch import autograd
import random
!pip install ray
!pip install torchmetrics
from ray import tune
from ray.tune import CLIReporter
import string
exclude = string.punctuation

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

from nltk import word_tokenize, pos_tag
from collections import Counter
from sklearn.preprocessing import LabelEncoder

torch.manual_seed(1)


import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 8.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.2 MB/s 
     |████████████████████████████████| 8.8 MB 52.9 MB/s 
     |████████████████████████████████| 468 kB 71.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
nlp = spacy.blank("en")
%matplotlib inline

In [4]:
# use the GPU
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


Importing and converting data into dataframe

In [5]:
!git clone https://github.com/cardiffnlp/tweeteval.git

Cloning into 'tweeteval'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 370 (delta 13), reused 1 (delta 1), pack-reused 354
Receiving objects: 100% (370/370), 8.49 MiB | 13.89 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [6]:
def load_text(path):
  
    with open(path, 'rb') as f:
        texts = []
        for line in f:
            texts.append(line.decode(errors='ignore').lower().strip())

    return texts

In [7]:
train_text = load_text("/content/tweeteval/datasets/emotion/train_text.txt")
train_labels = load_text("/content/tweeteval/datasets/emotion/train_labels.txt")
test_text = load_text("/content/tweeteval/datasets/emotion/test_text.txt")
test_labels = load_text("/content/tweeteval/datasets/emotion/test_labels.txt")
val_text = load_text("/content/tweeteval/datasets/emotion/val_text.txt")
val_labels = load_text("/content/tweeteval/datasets/emotion/val_labels.txt")

In [8]:
train_data = list(zip(train_text,train_labels))
test_data = list(zip(test_text,test_labels))
val_data = list(zip(val_text,val_labels))

df_train = pd.DataFrame(train_data,columns=["tweet","label"])
df_test = pd.DataFrame(test_data,columns=["tweet","label"])
df_val = pd.DataFrame(val_data,columns=["tweet","label"])

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3257 entries, 0 to 3256
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   3257 non-null   object
 1   label   3257 non-null   object
dtypes: object(2)
memory usage: 51.0+ KB


1. Dataframe 1 includes labels 0 and 1 (anger and joy)
2. Dataframe 2 includes labels 0 and 3 (anger and sadness)

In [10]:

print('There are {} tweets labeled 0'.format(len(df_train[df_train["label"].isin(['0'])])))
print('There are {} tweets labeled 1'.format(len(df_train[df_train["label"].isin(['1'])])))
print('There are {} tweets labeled 2'.format(len(df_train[df_train["label"].isin(['2'])])))
print('There are {} tweets labeled 3'.format(len(df_train[df_train["label"].isin(['3'])])))

# Dataframe 1 - labels 0 and 1
df_train_1 = df_train[df_train["label"].isin(['0','1'])]
df_test_1 = df_test[df_test["label"].isin(['0','1'])]
df_val_1 = df_val[df_val["label"].isin(['0','1'])]

# Dataframe 2 - labels 0 and 3
df_train_2 = df_train[df_train["label"].isin(['0','3'])]
df_test_2 = df_test[df_test["label"].isin(['0','3'])]
df_val_2 = df_val[df_val["label"].isin(['0','3'])]


There are 1400 tweets labeled 0
There are 708 tweets labeled 1
There are 294 tweets labeled 2
There are 855 tweets labeled 3


Data Preprocessing

In [11]:
# data preprocessing class
class Preprocessing:

  # lowercasing
  def convert_lowercase(self,text):
      text = text.lower()
      return text

  # removing html tags
  def remove_html_tags(self,text):
      re_html = re.compile('<.*?>')
      return re_html.sub(r'', text)

  # removing URLS
  def remove_url(self,text):
      re_url = re.compile('https?://\S+|www\.\S+')
      return re_url.sub('', text)

  # removing punctuations
  def remove_punc(self,text):
      return text.translate(str.maketrans('', '', exclude))

  # removing special characters
  def remove_special(self,text):
      x=''
      for i in text:
          if i.isalnum():
              x=x+i
          else:
              x=x+' '
      return x

  # removing digits
  def remove_digits(self,text):
      filtered_string = ''.join((x for x in text if not x.isdigit()))
      return filtered_string

  # removing stop words
  def remove_stopwords(self,text):
      new_text = []
      for word in text.split():
          if word in stopwords_english:
              continue
          else:
              new_text.append(word)

      return ' '.join(new_text)

  def preprocess(self,text):
    full_text = []
    for sent in text:
      sent = self.convert_lowercase(sent)
      sent = self.remove_html_tags(sent)
      sent = self.remove_url(sent)
      sent = self.remove_digits(sent)
      sent = self.remove_punc(sent)
      sent = self.remove_special(sent)
      sent = self.remove_stopwords(sent)
      full_text.append(sent)
    return full_text

Data tokentization and encoding

In [12]:
# function for text tokentization

def tokenize(texts):
  max_len = 0
  tokenized_texts = []
  word2idx = {}

  # Add <pad> and <unk> tokens to the vocabulary
  word2idx['<pad>'] = 0
  word2idx['<unk>'] = 1

  # Building our vocab from the corpus starting from index 2
  idx = 2
  for sent in texts:
    tokenized_sent = nlp(sent)
    # Add `tokenized_sent` to `tokenized_texts`
    tokenized_texts.append(tokenized_sent)
    # Add new token to `word2idx`
    for token in tokenized_sent:
      # string any token objects are different things, be careful.
      if token.text not in word2idx:
        word2idx[token.text] = idx
        idx += 1

        # Update `max_len`
    max_len = max(max_len, len(tokenized_sent))

  return tokenized_texts, word2idx, max_len

In [13]:
# function for converting words into numeric ids 
def encode(tokenized_texts, word2idx, max_len):
    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_padded_sent = list(tokenized_sent) + ['<pad>'] * (max_len - len(tokenized_sent))
 
        # Encode tokens to input_ids, assign value 1 if word doesn't exist in vocabulary
        input_id = [word2idx.get(str(token)) if str(token) in word2idx else 1 for token in tokenized_padded_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

Model

In [14]:
class CNN(nn.Module):
    def __init__(self,
                 vocab_size,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
      
        super(CNN, self).__init__()
        # Random Embedding layer
        self.embed_dim = embed_dim
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [23]:
# Our model class have preprocessing, encoding, data loader and train_model functions
class model_emotions:
  def __init__(self):
    pass

  # Takes training data and returns the processed tensor of input data and its labels 
  # Also returns the word2idx dictionary for validation and test data preprocessing
  def get_train_tokenizors(self,train_data):
    self.train_data = train_data
    preprocessed_text_train = Preprocessing().preprocess(self.train_data["tweet"])
    tokenized_texts, word2idx, max_len = tokenize(preprocessed_text_train)
    input_ids_train = encode(tokenized_texts, word2idx, max_len)

    # train label encoders
    label_encoder = LabelEncoder()
    train_data_labels = label_encoder.fit_transform(self.train_data["label"])


    #convert train inputs and labels to torch tensor

    train_inputs = torch.from_numpy(input_ids_train)
    labels_train = torch.from_numpy(train_data_labels)

    return word2idx, max_len, train_inputs, labels_train

  # returns the validation or test tensors
  def get_processed_data(self,data,word2idx,max_len):
    # preprocess the training and the validation data- learn the vocabulary from the training data and only encode the validation data   
    preprocessed_text = Preprocessing().preprocess(data["tweet"])
    input_ids = encode([nlp(sent) for sent in preprocessed_text], word2idx, max_len)

    # label encoders

    label_encoder = LabelEncoder()
    data_labels = label_encoder.fit_transform(data["label"])

    # Convert data type to torch.Tensor for input_ids and data_labels

    inputs = torch.from_numpy(input_ids)
    labels_data = torch.from_numpy(data_labels)

    return inputs, labels_data

  # takes the tensors and returns the DataLoader of training data
  def get_dataloader(self,train_data_tensor,labels_train_tensor,batch_size):

    # Specify batch_size
    self.batch_size = batch_size

    # Create DataLoader for training data
    train_data = TensorDataset(train_data_tensor, labels_train_tensor)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    return train_dataloader

  # train the model
  def train_model(self,config):

    train_data = config["train_data"]
    val_data = config["val_data"]


    # get the tensorial forms of the train and validation data
    word2idx,max_len,inputs_train,labels_train = self.get_train_tokenizors(train_data)
    inputs_val, labels_val = self.get_processed_data(val_data,word2idx,max_len)

    batch_size = config["batch_size"]


    model = CNN(vocab_size = len(word2idx),
                 embed_dim = 300,
                 filter_sizes = config["filter_sizes"],
                 num_filters = config["num_filters"],
                 num_classes = 2,
                 dropout = config["dropout"])


    model = model.to(device)

    # testing Adam and Adadelta optimizers

    # optimizer = optim.Adadelta(model.parameters(),
    #                            lr = config["lr"],
    #                            rho = 0.95)


    optimizer = optim.Adam(model.parameters(),
                              lr = config["lr"])
   

    loss_fn = nn.CrossEntropyLoss()

    # get the dataloader
    train_dataloader = self.get_dataloader(inputs_train,labels_train,batch_size)

    train_loss_per_epoch = []
    val_loss_per_epoch = []

    epochs = 10
    mean_val_acc = 0

    # train the model on the training data
    for epoch_i in range(epochs):
      total_loss = 0
      # Put the model into the training mode
      model.train()
      for step, batch in enumerate(train_dataloader):
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)
        b_labels = b_labels.type(torch.LongTensor)
        b_input_ids = b_input_ids.to(device)
        b_labels = b_labels.to(device)
        # Zero out any previously calculated gradients
        model.zero_grad()
        # Perform a forward pass. This will return logits.
        logits = model(b_input_ids)
        # Compute loss and accumulate the loss values
        loss = loss_fn(logits, b_labels)
        total_loss += loss.item()
        # Perform a backward pass to calculate gradients
        loss.backward()
        # Update parameters
        optimizer.step()

      # Calculate the average loss over the entire training data
      avg_train_loss = total_loss / len(train_dataloader)
      train_loss_per_epoch.append(avg_train_loss)

      # test the model on the validation data
      model.eval()
      val_data, val_labels = inputs_val.to(device), labels_val.type(torch.LongTensor).to(device)
      val_labels_pred = model(val_data)
      val_loss = loss_fn(val_labels_pred,val_labels)
      val_loss = val_loss.item()
      val_loss_per_epoch.append(val_loss)

      # validation accuracy
      val_accuracy = (torch.argmax(val_labels_pred, dim = 1) == val_labels).sum().item() / len(val_labels)
      mean_val_acc += val_accuracy / epochs 

      #f1 score on validation data
      f1 = BinaryF1Score(num_classes=2).to(device)
      f1_score_val = f1(torch.argmax(val_labels_pred, dim=1), val_labels)

    # adding this to configuration as ray doesn't work if function returns values
    # we only need validation loss and accuracy for hyperparameter tuning
    if config["train"]==True:
      tune.report(loss=val_loss, accuracy=mean_val_acc,f1_score =f1_score_val.item())  #this will send validation loss and accuracy to Tune
    else:

      return word2idx, max_len, model  

In [25]:

import torch.optim as optim

config = {"train_data":df_train_1,"val_data":df_val_1,
          "batch_size":tune.grid_search([8,16,24]),
          "filter_sizes":[3,4,5],
          "num_filters":tune.grid_search([[100,100,100],[150,150,150],[200,200,200]]),
          "dropout":tune.choice([0.5,0.6,0.8]),
          "lr": tune.grid_search([0.1, 0.001, 0.01]),
          "train": True
          }

reporter = CLIReporter(
        metric_columns=["accuracy", "loss", "f1_score"])

##### getting the best hyperparameters for label 0 and 1 model ######
analysis = tune.run(model_emotions().train_model,
                    config=config,
                    verbose=3,
                    resources_per_trial = {'gpu': 1, 'cpu': 2 },
                    progress_reporter = reporter,
                    resume = 'AUTO',
                    )

print("Best config: ", analysis.get_best_config("accuracy", "max"))

# Get a dataframe for analyzing trial results.
df1 = analysis.dataframe()

2022-11-14 01:29:27,880	INFO registry.py:96 -- Detected unknown callable for trainable. Converting to class.
2022-11-14 01:29:27,918	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2022-11-14 01:29:27,922	INFO trial_runner.py:597 -- No local checkpoint was found. Ray Tune will now start a new experiment.


== Status ==
Current time: 2022-11-14 01:29:30 (running for 00:00:02.78)
Memory usage on this node: 3.5/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/train_model_2022-11-14_01-29-27
Number of trials: 16/27 (15 PENDING, 1 RUNNING)
+-------------------------+----------+-----------------+--------------+-----------+-------+-----------------+
| Trial name              | status   | loc             |   batch_size |   dropout |    lr | num_filters     |
|-------------------------+----------+-----------------+--------------+-----------+-------+-----------------|
| train_model_c7b90_00000 | RUNNING  | 172.28.0.2:2230 |            8 |       0.5 | 0.1   | [100, 100, 100] |
| train_model_c7b90_00001 | PENDING  |                 |           16 |       0.8 | 0.1   | [100, 100, 100] |
| train_model_c7b90_00002 | PENDING  |                 |           24 |    

Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,f1_score,hostname,iterations_since_restore,loss,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_model_c7b90_00000,0.703113,2022-11-14_01-29-53,True,,0b9f4ac5757041199d05a3bdad9cea81,"0_batch_size=8,dropout=0.5000,lr=0.1000,num_filters=100_100_100",0.571429,234d23669228,1,1679.77,172.28.0.2,2230,12.865,12.865,12.865,1668389393,0,,1,c7b90_00000,0.0069437
train_model_c7b90_00001,0.718677,2022-11-14_01-30-12,True,,7cbbb70776284b85986e04e11c428fb0,"1_batch_size=16,dropout=0.8000,lr=0.1000,num_filters=100_100_100",0.630952,234d23669228,1,779.421,172.28.0.2,2276,7.92202,7.92202,7.92202,1668389412,0,,1,c7b90_00001,0.00646544
train_model_c7b90_00002,0.715564,2022-11-14_01-30-31,True,,b0af91ca810943e1ab6f62baed19cdfc,"2_batch_size=24,dropout=0.6000,lr=0.1000,num_filters=100_100_100",0.569697,234d23669228,1,758.605,172.28.0.2,2318,6.09862,6.09862,6.09862,1668389431,0,,1,c7b90_00002,0.00721717
train_model_c7b90_00003,0.754475,2022-11-14_01-30-54,True,,aa6b4d7dc7c645a18a59bb44e2d7ae7c,"3_batch_size=8,dropout=0.5000,lr=0.0010,num_filters=100_100_100",0.715517,234d23669228,1,1.09064,172.28.0.2,2362,12.3808,12.3808,12.3808,1668389454,0,,1,c7b90_00003,0.00860453
train_model_c7b90_00004,0.712451,2022-11-14_01-31-14,True,,bb5dbb68a6f54c8a81e6eb05f4ce3dc9,"4_batch_size=16,dropout=0.5000,lr=0.0010,num_filters=100_100_100",0.467153,234d23669228,1,1.41725,172.28.0.2,2406,7.72207,7.72207,7.72207,1668389474,0,,1,c7b90_00004,0.00751185
train_model_c7b90_00005,0.737743,2022-11-14_01-31-30,True,,b993a1e0bc954272ae2bdd2afe6f6127,"5_batch_size=24,dropout=0.5000,lr=0.0010,num_filters=100_100_100",0.550336,234d23669228,1,0.93604,172.28.0.2,2450,6.0727,6.0727,6.0727,1668389490,0,,1,c7b90_00005,0.00692153
train_model_c7b90_00006,0.765759,2022-11-14_01-31-55,True,,778b631ad9444f8ab751172d97702644,"6_batch_size=8,dropout=0.5000,lr=0.0100,num_filters=100_100_100",0.729858,234d23669228,1,26.2009,172.28.0.2,2494,14.0293,14.0293,14.0293,1668389515,0,,1,c7b90_00006,0.00718021
train_model_c7b90_00007,0.7607,2022-11-14_01-32-14,True,,036ce6344bfb4d12bede9f2ec141cbbc,"7_batch_size=16,dropout=0.6000,lr=0.0100,num_filters=100_100_100",0.700565,234d23669228,1,18.7058,172.28.0.2,2539,7.74432,7.74432,7.74432,1668389534,0,,1,c7b90_00007,0.00763941
train_model_c7b90_00008,0.772374,2022-11-14_01-32-31,True,,0da94b7194a2491a83cb433e77444fc8,"8_batch_size=24,dropout=0.5000,lr=0.0100,num_filters=100_100_100",0.746114,234d23669228,1,10.7339,172.28.0.2,2582,6.08551,6.08551,6.08551,1668389551,0,,1,c7b90_00008,0.00735259
train_model_c7b90_00009,0.731128,2022-11-14_01-32-57,True,,6dd728140fbf4710baa5256dec033fe9,"9_batch_size=8,dropout=0.8000,lr=0.1000,num_filters=150_150_150",0.724324,234d23669228,1,2110.18,172.28.0.2,2626,14.1695,14.1695,14.1695,1668389577,0,,1,c7b90_00009,0.00663686


== Status ==
Current time: 2022-11-14 01:29:59 (running for 00:00:32.00)
Memory usage on this node: 3.8/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/train_model_2022-11-14_01-29-27
Number of trials: 18/27 (16 PENDING, 1 RUNNING, 1 TERMINATED)
+-------------------------+------------+-----------------+--------------+-----------+-------+-----------------+------------+---------+------------+
| Trial name              | status     | loc             |   batch_size |   dropout |    lr | num_filters     |   accuracy |    loss |   f1_score |
|-------------------------+------------+-----------------+--------------+-----------+-------+-----------------+------------+---------+------------|
| train_model_c7b90_00001 | RUNNING    | 172.28.0.2:2276 |           16 |       0.8 | 0.1   | [100, 100, 100] |            |         |            |
| train_model_c7b90

2022-11-14 01:38:40,679	INFO tune.py:778 -- Total run time: 552.80 seconds (552.32 seconds for the tuning loop).


== Status ==
Current time: 2022-11-14 01:38:40 (running for 00:09:12.62)
Memory usage on this node: 3.6/12.7 GiB 
Using FIFO scheduling algorithm.
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.32 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/train_model_2022-11-14_01-29-27
Number of trials: 27/27 (27 TERMINATED)
+-------------------------+------------+-----------------+--------------+-----------+-------+-----------------+------------+-------------+------------+
| Trial name              | status     | loc             |   batch_size |   dropout |    lr | num_filters     |   accuracy |        loss |   f1_score |
|-------------------------+------------+-----------------+--------------+-----------+-------+-----------------+------------+-------------+------------|
| train_model_c7b90_00000 | TERMINATED | 172.28.0.2:2230 |            8 |       0.5 | 0.1   | [100, 100, 100] |   0.703113 | 1679.77     |   0.571429 |
| train_model_c7b90_00001 | T

In [26]:
# model with the best parameters for dataset 1
word2idx,max_len,label_01_model = model_emotions().train_model(config = {"train_data":df_train_1,"val_data":df_val_1,
                                                                        "batch_size": 8,
                                                                        "filter_sizes":[3,4,5],
                                                                        "num_filters":[150,150,150],
                                                                        "dropout":0.6,
                                                                        "lr": 0.001,
                                                                         "train":False})

# test the data
test_data_processed,test_labels = model_emotions().get_processed_data(df_test_1, word2idx, max_len)
test_data_processed = test_data_processed.to(device)
test_labels = test_labels.to(device)
test_label_predicted = label_01_model(test_data_processed)

# Accuracy of model on test_data
accuracy = torch.mean((torch.argmax(test_label_predicted, dim=1) == test_labels).float())
print("The test accuracy of the model is {}".format(accuracy))

# F1-Macro on test_data
f1 = BinaryF1Score(num_classes=2).to(device)
f1_score_model = f1(torch.argmax(test_label_predicted, dim=1), test_labels)
print("The f1-score of the model is {}".format(f1_score_model))

The test accuracy of the model is 0.7914847135543823
The f1-score of the model is 0.701095461845398


In [27]:
# model with the best parameters for dataset 2
word2idx,max_len,label_02_model = model_emotions().train_model(config = {"train_data":df_train_2,"val_data":df_val_2,
                                                                        "batch_size": 8,
                                                                        "filter_sizes":[3,4,5],
                                                                        "num_filters":[150,150,150],
                                                                        "dropout":0.5,
                                                                        "lr": 0.001,
                                                                         "train": False})

# test the data
test_data_processed,test_labels = model_emotions().get_processed_data(df_test_2, word2idx, max_len)
test_data_processed = test_data_processed.to(device)
test_labels = test_labels.to(device)
test_label_predicted = label_02_model(test_data_processed)

# Accuracy of model on test_data
accuracy = torch.mean((torch.argmax(test_label_predicted, dim=1) == test_labels).float())
print("The test accuracy of the model is {}".format(accuracy))

# F1-Macro on test_data
f1 = BinaryF1Score(num_classes=2).to(device)
f1_score_model = f1(torch.argmax(test_label_predicted, dim=1), test_labels)
print("The f1-score of the model is {}".format(f1_score_model))

The test accuracy of the model is 0.806382954120636
The f1-score of the model is 0.7553763389587402
